In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import pickle
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Subset
from torchvision.datasets import ImageNet, CIFAR10

from PyTorch_CIFAR10.cifar10_models.mobilenetv2 import mobilenet_v2
from PyTorch_CIFAR10.UTILS_TORCH import *

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Currently using: '{device}'")

Currently using: 'cuda'


In [3]:
# Hyperparameters
NUM_EPOCHS = 100
BATCH_SIZE = 64
LEARN_RATE = 0.001
ALPHA = 0.1
TEMP = 3
NUM_WORKERS = 16
OVERLAY_PROB = 0.25

In [4]:
IGS = "PyTorch_CIFAR10/Captum_IGs.npy"
print(f"Using IGs: {IGS}")

# Load the precomputed IGs
igs = np.load(IGS)
print(f"IGs shape: {igs.shape}")

Using IGs: PyTorch_CIFAR10/Captum_IGs.npy
IGs shape: (50000, 32, 32)


In [5]:
precomputed_logits = np.load("PyTorch_CIFAR10/data/cifar10_logits.npy")
print("Shape of teacher_logits:", precomputed_logits.shape)

Shape of teacher_logits: (50000, 10)


In [6]:
transform = transforms.Compose(
    [
        transforms.ToTensor(),
    ]
)

student_aug = transforms.Compose(
    [
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ]
)

In [7]:
train_dataset = CIFAR10WithIG(
    igs=igs,
    root="PyTorch_CIFAR10/data",
    train=True,
    transform=student_aug,
    precomputed_logits=precomputed_logits,
    overlay_prob=OVERLAY_PROB,
    return_ig=False,
)

# Load the data into batches
train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=NUM_WORKERS,
    pin_memory=False,
    persistent_workers=True,
)

# load student data
test_data = CIFAR10(
    root="PyTorch_CIFAR10/data",
    train=False,
    transform=transform,
)

test_loader = DataLoader(
    test_data,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
    pin_memory=False,
    persistent_workers=True,
)

Files already downloaded and verified


In [8]:
Teacher = mobilenet_v2(pretrained=True)
Teacher.to(device)

MobileNetV2(
  (features): Sequential(
    (0): ConvBNReLU(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=Tr

In [9]:
teacher_params = count_parameters(Teacher)

In [10]:
Student = SmallerMobileNet(mobilenet_v2(pretrained=False))
Student.to(device)

SmallerMobileNet(
  (features): Sequential(
    (0): ConvBNReLU(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affi

In [11]:
student_params = count_parameters(Student)

In [12]:
teacher_params / student_params

4.115345410654685

# First Student

In [13]:
class Smaller(nn.Module):
    def __init__(self, original_model):
        super(Smaller, self).__init__()
        self.features = nn.Sequential(*list(original_model.features.children())[:-3])

        # Dynamically find the output channels of the last convolutional block used
        # Iterate backwards over the blocks to find the last convolutional layer
        for block in reversed(self.features):
            if hasattr(block, "conv"):
                # If the block contains a 'conv' attribute, likely to be a sequential module
                if hasattr(block.conv, "__iter__"):
                    # Find the last Conv2d module in the block
                    for layer in reversed(block.conv):
                        if isinstance(layer, nn.Conv2d):
                            num_output_channels = layer.out_channels
                            break
                    break
            elif isinstance(block, nn.Conv2d):
                num_output_channels = block.out_channels
                break

        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(num_output_channels, 10),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.pool(x)
        x = self.classifier(x)
        return x

In [14]:
Student_1 = Smaller(mobilenet_v2(pretrained=True))
Student_1.to(device);

In [15]:
S1 = count_parameters(Student_1)
teacher_params / S1

2.194111842040726

In [16]:
Student_1, acc_1 = train_eval_kd(
    student=Student_1,
    train_loader=train_loader,
    test_loader=test_loader,
    epochs=NUM_EPOCHS,
    lr=LEARN_RATE,
    TEMP=TEMP,
    ALPHA=ALPHA,
    device=device,
    csv_path="PyTorch_CIFAR10/Histories/Compression/Compression_of_2.19.csv",
)

Epoch 1/100: Loss: 1.4204,            Accuracy: 63.19%


Epoch 2/100: Loss: 1.1559,            Accuracy: 74.81%


Epoch 3/100: Loss: 1.0824,            Accuracy: 77.87%


Epoch 4/100: Loss: 1.0398,            Accuracy: 79.53%


Epoch 5/100: Loss: 1.0103,            Accuracy: 80.85%


...

Epoch 96/100: Loss: 0.6700,            Accuracy: 94.97%


Epoch 97/100: Loss: 0.6662,            Accuracy: 95.05%


Epoch 98/100: Loss: 0.6667,            Accuracy: 95.06%


Epoch 99/100: Loss: 0.6647,            Accuracy: 95.16%


Epoch 100/100: Loss: 0.6661,            Accuracy: 95.11%


In [18]:
print(f"Test Acc = {acc_1:.2f}%")

Test Acc = 91.90%


# Second Student

In [25]:
class Smaller(nn.Module):
    def __init__(self, original_model):
        super(Smaller, self).__init__()
        self.features = nn.Sequential(*list(original_model.features.children())[:-7])

        # Dynamically find the output channels of the last convolutional block used
        # Iterate backwards over the blocks to find the last convolutional layer
        for block in reversed(self.features):
            if hasattr(block, "conv"):
                # If the block contains a 'conv' attribute, likely to be a sequential module
                if hasattr(block.conv, "__iter__"):
                    # Find the last Conv2d module in the block
                    for layer in reversed(block.conv):
                        if isinstance(layer, nn.Conv2d):
                            num_output_channels = layer.out_channels
                            break
                    break
            elif isinstance(block, nn.Conv2d):
                num_output_channels = block.out_channels
                break

        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(num_output_channels, 10),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.pool(x)
        x = self.classifier(x)
        return x

In [26]:
Student_2 = Smaller(mobilenet_v2(pretrained=True))
Student_2.to(device);

In [27]:
S2 = count_parameters(Student_2)
teacher_params / S2

7.286700938902897

In [28]:
Student_2, acc_2 = train_eval_kd(
    student=Student_2,
    train_loader=train_loader,
    test_loader=test_loader,
    epochs=NUM_EPOCHS,
    lr=LEARN_RATE,
    TEMP=TEMP,
    ALPHA=ALPHA,
    device=device,
    csv_path="PyTorch_CIFAR10/Histories/Compression/Compression_of_7.29.csv",
)

Epoch 1/100: Loss: 1.4793,            Accuracy: 60.94%


Epoch 2/100: Loss: 1.1779,            Accuracy: 73.89%


Epoch 3/100: Loss: 1.1016,            Accuracy: 77.56%


Epoch 4/100: Loss: 1.0566,            Accuracy: 79.35%


Epoch 5/100: Loss: 1.0269,            Accuracy: 80.89%


...

Epoch 96/100: Loss: 0.6969,            Accuracy: 94.46%


Epoch 97/100: Loss: 0.6952,            Accuracy: 94.73%


Epoch 98/100: Loss: 0.6935,            Accuracy: 94.72%


Epoch 99/100: Loss: 0.6935,            Accuracy: 94.74%


Epoch 100/100: Loss: 0.6913,            Accuracy: 94.78%


In [29]:
print(f"Test Acc = {acc_2:.2f}%")

Test Acc = 92.19%


# Third Student

In [30]:
class Smaller(nn.Module):
    def __init__(self, original_model):
        super(Smaller, self).__init__()
        self.features = nn.Sequential(*list(original_model.features.children())[:-9])

        # Dynamically find the output channels of the last convolutional block used
        # Iterate backwards over the blocks to find the last convolutional layer
        for block in reversed(self.features):
            if hasattr(block, "conv"):
                # If the block contains a 'conv' attribute, likely to be a sequential module
                if hasattr(block.conv, "__iter__"):
                    # Find the last Conv2d module in the block
                    for layer in reversed(block.conv):
                        if isinstance(layer, nn.Conv2d):
                            num_output_channels = layer.out_channels
                            break
                    break
            elif isinstance(block, nn.Conv2d):
                num_output_channels = block.out_channels
                break

        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(num_output_channels, 10),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.pool(x)
        x = self.classifier(x)
        return x

In [31]:
Student_3 = Smaller(mobilenet_v2(pretrained=True))
Student_3.to(device);

In [32]:
S3 = count_parameters(Student_3)
teacher_params / S3

12.042134619733172

In [33]:
Student_3, acc_3 = train_eval_kd(
    student=Student_3,
    train_loader=train_loader,
    test_loader=test_loader,
    epochs=NUM_EPOCHS,
    lr=LEARN_RATE,
    TEMP=TEMP,
    ALPHA=ALPHA,
    device=device,
    csv_path="PyTorch_CIFAR10/Histories/Compression/Compression_of_12.04.csv",
)

Epoch 1/100: Loss: 1.5207,            Accuracy: 58.55%


Epoch 2/100: Loss: 1.2506,            Accuracy: 70.96%


Epoch 3/100: Loss: 1.1633,            Accuracy: 75.07%


Epoch 4/100: Loss: 1.1172,            Accuracy: 77.10%


Epoch 5/100: Loss: 1.0809,            Accuracy: 78.61%


...

Epoch 96/100: Loss: 0.7434,            Accuracy: 93.18%


Epoch 97/100: Loss: 0.7449,            Accuracy: 92.89%


Epoch 98/100: Loss: 0.7397,            Accuracy: 93.16%


Epoch 99/100: Loss: 0.7405,            Accuracy: 93.20%


Epoch 100/100: Loss: 0.7374,            Accuracy: 93.35%


In [34]:
print(f"Test Acc = {acc_3:.2f}%")

Test Acc = 90.66%


# Fourth Student

In [35]:
class Smaller(nn.Module):
    def __init__(self, original_model):
        super(Smaller, self).__init__()
        self.features = nn.Sequential(*list(original_model.features.children())[:-11])

        # Dynamically find the output channels of the last convolutional block used
        # Iterate backwards over the blocks to find the last convolutional layer
        for block in reversed(self.features):
            if hasattr(block, "conv"):
                # If the block contains a 'conv' attribute, likely to be a sequential module
                if hasattr(block.conv, "__iter__"):
                    # Find the last Conv2d module in the block
                    for layer in reversed(block.conv):
                        if isinstance(layer, nn.Conv2d):
                            num_output_channels = layer.out_channels
                            break
                    break
            elif isinstance(block, nn.Conv2d):
                num_output_channels = block.out_channels
                break

        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(num_output_channels, 10),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.pool(x)
        x = self.classifier(x)
        return x

In [36]:
Student_4 = Smaller(mobilenet_v2(pretrained=True))
Student_4.to(device);

In [37]:
S4 = count_parameters(Student_4)
teacher_params / S4

28.974816695598104

In [38]:
Student_4, acc_4 = train_eval_kd(
    student=Student_4,
    train_loader=train_loader,
    test_loader=test_loader,
    epochs=NUM_EPOCHS,
    lr=LEARN_RATE,
    TEMP=TEMP,
    ALPHA=ALPHA,
    device=device,
    csv_path="PyTorch_CIFAR10/Histories/Compression/Compression_of_28.97.csv",
)

Epoch 1/100: Loss: 1.6506,            Accuracy: 53.28%


Epoch 2/100: Loss: 1.3299,            Accuracy: 67.82%


Epoch 3/100: Loss: 1.2331,            Accuracy: 72.49%


Epoch 4/100: Loss: 1.1856,            Accuracy: 74.86%


Epoch 5/100: Loss: 1.1490,            Accuracy: 76.66%


...

Epoch 96/100: Loss: 0.8757,            Accuracy: 89.91%


Epoch 97/100: Loss: 0.8760,            Accuracy: 89.90%


Epoch 98/100: Loss: 0.8741,            Accuracy: 89.84%


Epoch 99/100: Loss: 0.8740,            Accuracy: 89.80%


Epoch 100/100: Loss: 0.8742,            Accuracy: 89.96%


In [39]:
print(f"Test Acc = {acc_4:.2f}%")

Test Acc = 87.81%


# Fifth Student

In [40]:
class Smaller(nn.Module):
    def __init__(self, original_model):
        super(Smaller, self).__init__()
        self.features = nn.Sequential(*list(original_model.features.children())[:-13])

        # Dynamically find the output channels of the last convolutional block used
        # Iterate backwards over the blocks to find the last convolutional layer
        for block in reversed(self.features):
            if hasattr(block, "conv"):
                # If the block contains a 'conv' attribute, likely to be a sequential module
                if hasattr(block.conv, "__iter__"):
                    # Find the last Conv2d module in the block
                    for layer in reversed(block.conv):
                        if isinstance(layer, nn.Conv2d):
                            num_output_channels = layer.out_channels
                            break
                    break
            elif isinstance(block, nn.Conv2d):
                num_output_channels = block.out_channels
                break

        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(num_output_channels, 10),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.pool(x)
        x = self.classifier(x)
        return x

In [41]:
Student_5 = Smaller(mobilenet_v2(pretrained=True))
Student_5.to(device);

S5 = count_parameters(Student_5)
teacher_params / S5

54.5931657310227

In [42]:
Student_5, acc_5 = train_eval_kd(
    student=Student_5,
    train_loader=train_loader,
    test_loader=test_loader,
    epochs=NUM_EPOCHS,
    lr=LEARN_RATE,
    TEMP=TEMP,
    ALPHA=ALPHA,
    device=device,
    csv_path="PyTorch_CIFAR10/Histories/Compression/Compression_of_54.59.csv",
)

Epoch 1/100: Loss: 1.7724,            Accuracy: 46.36%


Epoch 2/100: Loss: 1.4658,            Accuracy: 61.04%


Epoch 3/100: Loss: 1.3782,            Accuracy: 65.47%


Epoch 4/100: Loss: 1.3289,            Accuracy: 67.87%


Epoch 5/100: Loss: 1.2860,            Accuracy: 70.16%


...

Epoch 96/100: Loss: 0.9751,            Accuracy: 85.92%


Epoch 97/100: Loss: 0.9761,            Accuracy: 85.66%


Epoch 98/100: Loss: 0.9748,            Accuracy: 85.74%


Epoch 99/100: Loss: 0.9789,            Accuracy: 85.51%


Epoch 100/100: Loss: 0.9760,            Accuracy: 85.79%


In [43]:
print(f"Test Acc = {acc_5:.2f}%")

Test Acc = 84.68%


# Sixth Student

In [44]:
class Smaller(nn.Module):
    def __init__(self, original_model):
        super(Smaller, self).__init__()
        self.features = nn.Sequential(*list(original_model.features.children())[:-15])

        # Dynamically find the output channels of the last convolutional block used
        # Iterate backwards over the blocks to find the last convolutional layer
        for block in reversed(self.features):
            if hasattr(block, "conv"):
                # If the block contains a 'conv' attribute, likely to be a sequential module
                if hasattr(block.conv, "__iter__"):
                    # Find the last Conv2d module in the block
                    for layer in reversed(block.conv):
                        if isinstance(layer, nn.Conv2d):
                            num_output_channels = layer.out_channels
                            break
                    break
            elif isinstance(block, nn.Conv2d):
                num_output_channels = block.out_channels
                break

        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(num_output_channels, 10),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.pool(x)
        x = self.classifier(x)
        return x

In [45]:
Student_6 = Smaller(mobilenet_v2(pretrained=True))
Student_6.to(device);

S6 = count_parameters(Student_6)
teacher_params / S6

139.426630095998

In [48]:
Student_6, acc_6 = train_eval_kd(
    student=Student_6,
    train_loader=train_loader,
    test_loader=test_loader,
    epochs=NUM_EPOCHS,
    lr=LEARN_RATE,
    TEMP=TEMP,
    ALPHA=ALPHA,
    device=device,
    csv_path="PyTorch_CIFAR10/Histories/Compression/Compression_of_139.43.csv",
)

Epoch 1/100: Loss: 1.9324,            Accuracy: 37.22%


Epoch 2/100: Loss: 1.6509,            Accuracy: 52.24%


Epoch 3/100: Loss: 1.5302,            Accuracy: 58.24%


Epoch 4/100: Loss: 1.4751,            Accuracy: 61.23%


Epoch 5/100: Loss: 1.4382,            Accuracy: 63.05%


...

Epoch 96/100: Loss: 1.1760,            Accuracy: 76.65%


Epoch 97/100: Loss: 1.1773,            Accuracy: 76.30%


Epoch 98/100: Loss: 1.1782,            Accuracy: 76.34%


Epoch 99/100: Loss: 1.1769,            Accuracy: 76.51%


Epoch 100/100: Loss: 1.1772,            Accuracy: 76.35%


In [49]:
print(f"Test Acc = {acc_6:.2f}%")

Test Acc = 76.68%


# Seventh Student

In [50]:
class Smaller(nn.Module):
    def __init__(self, original_model):
        super(Smaller, self).__init__()
        self.features = nn.Sequential(*list(original_model.features.children())[:-17])

        # Dynamically find the output channels of the last convolutional block used
        # Iterate backwards over the blocks to find the last convolutional layer
        for block in reversed(self.features):
            if hasattr(block, "conv"):
                # If the block contains a 'conv' attribute, likely to be a sequential module
                if hasattr(block.conv, "__iter__"):
                    # Find the last Conv2d module in the block
                    for layer in reversed(block.conv):
                        if isinstance(layer, nn.Conv2d):
                            num_output_channels = layer.out_channels
                            break
                    break
            elif isinstance(block, nn.Conv2d):
                num_output_channels = block.out_channels
                break

        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(num_output_channels, 10),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.pool(x)
        x = self.classifier(x)
        return x

In [51]:
Student_7 = Smaller(mobilenet_v2(pretrained=True))
Student_7.to(device);

S7 = count_parameters(Student_7)
teacher_params / S7

1121.7061183550652

In [52]:
Student_7, acc_7 = train_eval_kd(
    student=Student_7,
    train_loader=train_loader,
    test_loader=test_loader,
    epochs=NUM_EPOCHS,
    lr=LEARN_RATE,
    TEMP=TEMP,
    ALPHA=ALPHA,
    device=device,
    csv_path="PyTorch_CIFAR10/Histories/Compression/Compression_of_1121.71.csv",
)

Epoch 1/100: Loss: 2.1479,            Accuracy: 25.03%


Epoch 2/100: Loss: 1.9805,            Accuracy: 32.91%


Epoch 3/100: Loss: 1.9491,            Accuracy: 35.10%


Epoch 4/100: Loss: 1.9235,            Accuracy: 36.97%


Epoch 5/100: Loss: 1.8908,            Accuracy: 39.16%


...

Epoch 96/100: Loss: 1.6971,            Accuracy: 50.68%


Epoch 97/100: Loss: 1.6954,            Accuracy: 50.81%


Epoch 98/100: Loss: 1.6957,            Accuracy: 50.84%


Epoch 99/100: Loss: 1.6939,            Accuracy: 51.04%


Epoch 100/100: Loss: 1.6944,            Accuracy: 50.72%


In [53]:
print(f"Test Acc = {acc_7:.2f}%")

Test Acc = 48.82%


# Normal Student

In [64]:
Student_8 = SmallerMobileNet(mobilenet_v2(pretrained=True))
Student_8.to(device)

S8 = count_parameters(Student_8)
teacher_params / S8

4.115345410654685

In [65]:
Student_8, acc_8 = train_eval_kd(
    student=Student_8,
    train_loader=train_loader,
    test_loader=test_loader,
    epochs=NUM_EPOCHS,
    lr=LEARN_RATE,
    TEMP=TEMP,
    ALPHA=ALPHA,
    device=device,
    csv_path="PyTorch_CIFAR10/Histories/Compression/Compression_of_4.12.csv",
)

Epoch 1/100: Loss: 1.4489,            Accuracy: 61.13%


Epoch 2/100: Loss: 1.1754,            Accuracy: 74.23%


Epoch 3/100: Loss: 1.0974,            Accuracy: 77.54%


Epoch 4/100: Loss: 1.0515,            Accuracy: 79.60%


Epoch 5/100: Loss: 1.0192,            Accuracy: 80.84%


...

Epoch 96/100: Loss: 0.6615,            Accuracy: 95.35%


Epoch 97/100: Loss: 0.6634,            Accuracy: 95.30%


Epoch 98/100: Loss: 0.6608,            Accuracy: 95.39%


Epoch 99/100: Loss: 0.6605,            Accuracy: 95.33%


Epoch 100/100: Loss: 0.6578,            Accuracy: 95.48%


In [66]:
print(f"Test Acc = {acc_8:.2f}%")

Test Acc = 91.85%
